# 5장. GUI 원격 제어기로 인공지능 무인운반차량(AGV) 제어하기

유저는 GUI 원격 제어기의 버튼 제어부를 터치하여 명령을 publish 합니다.  
publish 된 명령어들은 인공지능 무인운반차량(AGV) 에서 subscribe 하여 동작합니다.

인공지능 무인운반차량(AGV)는 1초에 한번 씩, 랜덤 값을 생성하여 publish 합니다.  
publish 된 데이터들은 GUI 원격 제어기에서 subscribe 하여 sensingTable에 표시합니다.

## 라이브러리 가져오기

In [18]:
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg
from SCSCtrl import TTLServo

import threading

import paho.mqtt.client as mqtt
from datetime import datetime
import pytz
import time
import json
import random as rd

## 카메라, 로봇 객체 초기화 하기

In [19]:
robot = Robot()
camera = Camera()

## widget 생성하기

widget 은 간단하게 디자인하였습니다.

Start 버튼을 눌러서, AGV에서 GUI Controller 로 랜덤으로 생성한 데이터를 publish 합니다.  
manual 버튼을 누르면, GUI Controller 에서 버튼을 눌러서 AGV를 제어할 수 있습니다.

In [20]:
lbl1 = ipywidgets.Label(value="MQTT Test")

startBtn = widgets.Button(description='Start', button_style='info')

lbl2 = ipywidgets.Label(value="sensingData")
lbl3 = ipywidgets.Label(value="num1 : ")
num1lbl = ipywidgets.Label(value="0")
hbox1 = widgets.HBox([lbl3, num1lbl] )

lbl4 = ipywidgets.Label(value="num2 : ")
num2lbl = ipywidgets.Label(value="0")
hbox2 = widgets.HBox([lbl4, num2lbl] )
vbox1 = widgets.VBox([startBtn, lbl2, hbox1, hbox2])

manualBtn = widgets.Button(description='Manual', button_style='success', disabled = True)
lbl5 = ipywidgets.Label(value="commandData")
lbl6 = ipywidgets.Label(value="command : ")
commandlbl = ipywidgets.Label(value="None")
hbox3 = widgets.HBox([lbl6, commandlbl] )
vbox2 = widgets.VBox([manualBtn, lbl5, hbox3])

hbox4 = widgets.HBox([vbox1, vbox2] )

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

display(lbl1, hbox4, image_widget)

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Label(value='MQTT Test')

Image(value=b'', format='jpeg', height='224', width='224')

## MQTT Protocol을 위한 변수 설정하기

In [21]:
# 한국 시간대 (Asia/Seoul)로 설정
korea_timezone = pytz.timezone("Asia/Seoul")

#Broker IP Address 와 Port
address = "172.20.10.6"
port = 1883

commandTopic = "AGV/command"

sensingTopic = "AGV/sensing"
sensingData = {
    "time" : "None",
    "num1": 0.15,
    "num2": 0.99,
    "is_finish": 0,
    "manual_mode" : "off"
}

publishingData = None

## 인공지능 무인운반차량(AGV) 제어용 함수 정의하기

제어용 함수에 매개변수를 추가하여, arg_string 값으로 세부 동작을 제어할 수 있습니다.

In [22]:
def agv_stop():
    robot.stop()
    
def agv_forward():
    robot.forward(0.4)

def agv_backward():
    robot.backward(0.4)

def agv_left():
    robot.left(0.3)

def agv_right():
    robot.right(0.3)

## MQTT protocol을 위한 콜백함수 정의하기

MQTT protocol을 위한 콜백함수를 사용자가 새롭게 정의할 수 있습니다.  
MQTT protocol의 .loop_start() / .loop_stop() 을 이용하면, 내부적으로 thread 로 동작하여 비동기적으로 제어가 가능합니다.

In [23]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("connected OK")
    else:
        print("Bad connection Returned code=", rc)

def on_publish(client, userdata, result):
    print("data published")

def on_message(client, userdata, msg):
    global message
    message = json.loads(msg.payload.decode("utf-8"))
    #print(message, type(message))
    
    commandlbl.value = message["cmd_string"]
    if message["cmd_string"] == "go":      agv_forward()
    elif message["cmd_string"] == "mid":   agv_stop()
    elif message["cmd_string"] == "left":  agv_left()
    elif message["cmd_string"] == "right": agv_right()
    elif message["cmd_string"] == "back":  agv_backward()
    elif message["cmd_string"] == "stop":  print('stop')
    elif message["cmd_string"] == "exit":
        print('exit')
        publishingData.stop()
        agv_stop()

## sensorReadPublish() 클래스 생성하기

Start 버튼을 누르면, sensorReadPublish() 로 객체를 생성하여 비동기적으로 동작합니다.  
random 라이브러리를 이용하여, 변수 값을 만들고, 현재 시간 데이터와 함께 GUI Remote Controller 로 publish 합니다.

In [24]:
class sensorReadPublish(threading.Thread):
    
    def __init__(self):
        super().__init__()
        self.th_flag = True
        
    def run(self):

        while self.th_flag:
            num1 = round(rd.random(),2)
            num2 = round(rd.random(),2)
            num1lbl.value = str(num1)
            num2lbl.value = str(num2)
            
            current_time = datetime.now(korea_timezone)
            sensingData["time"] = current_time.strftime("%Y-%m-%d %H:%M:%S")
            sensingData["num1"] = num1
            sensingData["num2"] = num2
            
            client.publish(sensingTopic, json.dumps(sensingData), 1)
            time.sleep(0.5)
        
    def stop(self):
        self.th_flag = False


## Start 버튼용 함수 정의하고 바인딩하기

Start 버튼을 클릭하면, MQTT Client() 객체를 생성하고, Broker 에 연결을 시도합니다. 
sensorReadPublish() 객체를 생성하고, GUI Controller 로 data를 publish 합니다.  
Manual 버튼을 활성화하고, Start 버튼의 text 를 Stop으로 변경합니다.  

Stop 버튼을 클릭하면, data Publish 를 중단합니다.

In [25]:
client = None
def startPub(change):
    global publishingData, startBtn, client
    
    if startBtn.description == "Start" :
        client = mqtt.Client()
        client.connect(address, port)
        
        client.on_connect = on_connect
        client.on_publish = on_publish
        client.loop_start()
        
        publishingData = sensorReadPublish()
        publishingData.start()
        
        
        startBtn.description = "Stop"
        startBtn.button_style = 'danger'
        manualBtn.disabled = False
        
    elif startBtn.description == "Stop" :
        publishingData.stop()
        
        startBtn.description = "Start"
        startBtn.button_style = 'info'
        #print("Stop", client.is_connected())

startBtn.on_click(startPub)

## Manual 버튼용 함수 정의하고 바인딩하기
Manual 버튼을 클릭하면, GUI Controller 에서 보낸 데이터를 Subscribe 하게 됩니다.  
on_message 콜백 함수에서 정의한 것과 같이, GUI Controller 에서 버튼을 누르면 해당 명령어대로 AGV를 조종할 수 있습니다.

In [26]:
def startSub(change):
    global manualBtn, client
    
    if manualBtn.description == "Manual" :
        
        client.subscribe(commandTopic, 1)
        client.on_message = on_message
        
        manualBtn.description = "Auto"
        manualBtn.button_style = 'success'
        

    elif manualBtn.description == "Auto" :
        publishingData.stop()
        manualBtn.description = "Manual"
        manualBtn.button_style = 'warning'
        #print("Stop", client.is_connected())
        manualBtn.disabled = True

manualBtn.on_click(startSub)

## 프로젝트 종료하기

In [27]:
time.sleep(0.1)
robot.stop()
camera.stop()
print('End')

data published
End
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data p